In [1]:
import pandas as pd
import numpy as np
from buckaroo.buckaroo_widget import BuckarooWidget, disable, enable

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
df = pd.read_csv('/Users/paddy/code/citibike-play/2014-01 - Citi Bike trip data.csv')[:205]
#df
#df
#bw = BuckarooWidget(df[:10_000])
bw = BuckarooWidget(df)
bw

BuckarooWidget(commandConfig={'argspecs': {'dropcol': [None], 'to_datetime': [None], 'safeint': [None], 'filln…

In [3]:
bw.widgetConfig = {'showCommands': True, 'showTransformed': False}

In [ ]:
df

In [ ]:
ab = dict(a = 5, b =10)
ab.copy().update(dict(a=3, c=20))
ab

In [ ]:
df2 = df[:25000]

In [ ]:
df2

In [ ]:
from buckaroo.summary_stats import summarize_df

In [ ]:
sdf = summarize_df(df2)
#sdf

In [ ]:
pd.to_datetime(df['start station latitude'])

In [ ]:
ab = df['start station latitude']
ab.sample(500)

In [ ]:
np.min([1,5])

In [ ]:
pd.to_datetime('1971-01-01')

In [ ]:
pd.to_datetime(1_00_000_000_000_000_000)

In [ ]:
pd.to_datetime('1973-01-01')

In [ ]:
def probable_datetime(ser):
    s_ser = ser.sample(np.min([len(ser), 500]))
    try:
        dt_ser = pd.to_datetime(s_ser)
        #pd.to_datetime(1_00_000_000_000_000_000) == pd.to_datetime('1973-01-01') 
        if dt_ser.max() < pd.to_datetime('1973-01-01'):
            return False
        return True
        
    except Exception as e:
        return False
#probable_datetime(df['start station name'])
[[col, probable_datetime(df[col])] for col in df.columns]

In [ ]:
pd.to_datetime(df['start station name'])

In [ ]:
def numerize_column(ser):
    if pd.api.types.is_numeric_dtype(ser.dtype):
        return ser
    else:
        return make_num_categorical(ser)

In [ ]:

def without(arr, search_keys):
    new_arr = []
    for v in arr:
        if v not in search_keys:
            new_arr.append(v)
    return new_arr

def all_duplicates(raw_corr_pairs):
    flatlist = reduce(lambda a,b:a+b, raw_corr_pairs.values())
    return set(flatlist)


In [ ]:
def make_num_categorical(ser):
    ser_uniq = ser.unique()
    name_to_idx = {name:idx for idx, name in enumerate(ser_uniq)}
    #needs to be vectorized
    num_categorical = ser.apply(lambda x:name_to_idx[x])
    return num_categorical

def get_cor_pair_dict(df, summary_stats):
    #we need to remove columns with only nans or a single value, they mess up corr()

    #this needs to be vectorized
    corrable_cols = [col for col in df if summary_stats[col]['distinct_count'] > 1]
    #print("corrable_cols", corrable_cols)
    #num_df =  pd.DataFrame({col:numerize_column(df[col]) for col in corrable_cols})

    num_df =  pd.DataFrame({col:make_num_categorical(df[col]) for col in corrable_cols})

    corr_df = num_df.corr()
    high_corr_df = corr_df[corr_df > 0.99]
    cor_dict = {}
    for col in high_corr_df.columns:
        #columns with high correlation that aren't the column itself
        other_cor_cols = high_corr_df[col].dropna().drop(col)
        cor_cols = other_cor_cols.index.values
        if len(cor_cols) > 0:
            cor_dict[col] = cor_cols.tolist()
    return cor_dict


In [ ]:
ab = make_num_categorical(df2['start station id'])
bc = make_num_categorical(df2['start station name'])
cd = make_num_categorical(df2['start station latitude'])

In [ ]:
pd.DataFrame({'id':ab, 'name':bc, 'lat':cd, 'bikeid': make_num_categorical(df2['bikeid'])}).corr()

In [ ]:
get_cor_pair_dict(df2, sdf)

In [ ]:
cpd = get_cor_pair_dict(df2, sdf)

In [ ]:
def one_directional_dict(raw_corr_pair):
    # convert from two way links to only one way based on first seen
    seen = []
    ret_corr_pair = {}
    for key, other_key_list in raw_corr_pair.items():
        seen.append(key)
        stripped_other = without(other_key_list, seen)
        if len(stripped_other) > 0:
            ret_corr_pair[key] = stripped_other
    return ret_corr_pair


def all_duplicates(raw_corr_pairs):
    flatlist = reduce(lambda a,b:a+b, raw_corr_pairs.values())
    return set(flatlist)


In [ ]:
def find_groupings(corr_pairs):
    all_groupings = []
    for key, other_key_list in corr_pairs.items():
        ab = other_key_list.copy()
        ab.append(key)
        all_groupings.append(set(ab))
    return np.unique(all_groupings)

In [ ]:
groupings = find_groupings(cpd)
groupings

In [ ]:
def order_groupings(grps, ranked_cols):
    first_cols, rest_cols = [], []
    for col in ranked_cols:
        for grp in grps:
            if col in grp:
                first_cols.append(col)
                rest_cols.extend(list(without(grp, [col])))
                grps = without(grps, [grp])
    return first_cols, rest_cols


In [ ]:
order_groupings(groupings, ['start station name', 'end station name', 'stoptime', 'starttime'])

In [ ]:
one_directional_dict(cpd)

In [ ]:
def find_groupings(corr_pairs):
    all_groupings = []
    for key, other_key_list in corr_pairs.items():
        ab = other_key_list.copy()
        ab.append(key)
        all_groupings.append(set(ab))
    return np.unique(all_groupings)
def order_groupings(grps, ranked_cols):
    first_cols, rest_cols = [], []
    for col in ranked_cols:
        for grp in grps:
            if col in grp:
                first_cols.append(col)
                rest_cols.extend(list(without(grp, [col])))
                grps = without(grps, [grp])
    return first_cols, rest_cols

In [ ]:
from IPython.core.getipython import get_ipython
from IPython.display import display
ip = get_ipython()
ip_formatter = ip.display_formatter.ipython_display_formatter
ip_formatter.type_printers.pop(pd.DataFrame, None)

In [ ]:
pd.api.types.is_numeric_dtype(ser.dtype):

In [ ]:
pd.api.types.is_integer_dtype

In [ ]:
[pd.api.types.is_integer_dtype(df2[col]) for col in df2.columns]
[pd.api.types.is_numeric_dtype(df2[col]) for col in df2.columns]

In [ ]:
disable()

In [ ]:
sdf

In [ ]:
tsdf = sdf.copy()
tsdf.loc['first_col'] = 0
tsdf.loc['first_col':, ['starttime', 'start station name']] = 5
tsdf

In [ ]:

set_when(sdf.copy(), 'is_integer', 'grouping_score_integer', -3, 0)

#only_ones = (sdf.loc['distinct_count'] <= 1)
#    sdf.loc['one_distinct', only_ones[only_ones==True].index.values] = -20


In [ ]:
set_when(tsdf, 'is_integer', 'grouping_score_integer', -3, 0)
set_when(tsdf, 'is_numeric', 'grouping_score_numeric', -3, 5)
grouping_col_scores = tsdf.loc[['grouping_score_integer', 'grouping_score_numeric']].sum()
# col_scores.sort_values().index.values[::-1]

grouping_col_scores.sort_values()

In [ ]:
grouping_col_scores.sort_values().index[::-1].values

In [ ]:
from functools import reduce
def set_when(df, cond_row_name, target_row_name, true_val, false_val):
    true_row = df.loc[cond_row_name]
    df.loc[target_row_name] = false_val
    df.loc[target_row_name, true_row[true_row==True].index.values] = true_val
    return df

def all_duplicates(raw_corr_pairs):
    flatlist = reduce(lambda a,b:a+b, raw_corr_pairs.values())
    return set(flatlist)

def find_groupings(corr_pairs):
    all_groupings = []
    for key, other_key_list in corr_pairs.items():
        ab = other_key_list.copy()
        ab.append(key)
        all_groupings.append(set(ab))
    return np.unique(all_groupings)
def order_groupings(grps, ranked_cols):
    first_cols, rest_cols = [], []
    for col in ranked_cols:
        for grp in grps:
            if col in grp:
                first_cols.append(col)
                rest_cols.extend(list(without(grp, [col])))
                grps = without(grps, [grp])
    return first_cols, rest_cols

def order_columns(summary_stats_df, corr_pair_dict):
    sdf = summary_stats_df.copy()
    sdf.loc['one_distinct'] = 0

    only_ones = (sdf.loc['distinct_count'] <= 1)
    sdf.loc['one_distinct', only_ones[only_ones==True].index.values] = -20
    
    sdf.loc['first_col'] = 0
    sdf.loc['is_duplicate'] = 0
    
    set_when(sdf, 'is_integer', 'grouping_score_integer', -3, 0)
    set_when(sdf, 'is_numeric', 'grouping_score_numeric', -3, 5)
    grouping_col_scores = sdf.loc[['grouping_score_integer', 'grouping_score_numeric']].sum()
    duplicate_col_rankings = grouping_col_scores.sort_values().index[::-1].values

    groupings = find_groupings(corr_pair_dict)
    first_cols, duplicate_cols = order_groupings(groupings, duplicate_col_rankings)
    
    sdf.loc['first_col':, first_cols] = 5
    sdf.loc['is_duplicate':, duplicate_cols] = -5
    
    col_scores = sdf.loc[['one_distinct', 'first_col', 'is_duplicate']].sum()
    return col_scores.sort_values().index.values[::-1]

In [ ]:
order_columns(sdf, cpd)

In [ ]:
cpd = get_cor_pair_dict(df2, sdf)

In [ ]:
order_columns(sdf, cpd)

In [ ]:
from IPython.core.getipython import get_ipython

In [ ]:
from IPython.core.getipython import get_ipython
from IPython.display import display
ip = get_ipython()
ip_formatter = ip.display_formatter.ipython_display_formatter
def _display_as_buckaroo(df):
    return display(BuckarooWidget(df))
ip_formatter.for_type(pd.DataFrame, _display_as_buckaroo)

In [ ]:
def _display_as_buckaroo(df):
    return display(BuckarooWidget(df))
ip_formatter.for_type(pd.DataFrame, _display_as_buckaroo)

In [ ]:
ip_formatter.for_type(pd.DataFrame, _display_as_buckaroo)

In [ ]:
df

In [ ]:
display(BuckarooWidget(df))

In [ ]:
bw.dfConfig['sampled'] = False
bw

In [ ]:
def sample(df, sample_size=500, include_outliers=True):
    
    sample_size = np.min([sample_size, len(df)])
    sdf = df.sample(sample_size)
    
    
    if include_outliers:
        outlier_idxs = []
        for col in df.columns:
            idxs = df[col].sort_values().index
            outlier_idxs.extend(idxs[:5])
            outlier_idxs.extend(idxs[-5:])
        outlier_idxs.extend(sdf.index)
        uniq_idx = np.unique(outlier_idxs)
        #print(uniq_idx)
        return df.iloc[uniq_idx]
    return sdf


In [ ]:
%timeit sample(df, 500)

In [ ]:
%timeit sample(df, 10)

In [ ]:
%timeit sample(df, 5000)

In [ ]:
len(sample(df))

In [ ]:
df.iloc[[23,58, 1023]]

In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv('./examples/data/2014-01-citibike-tripdata.csv')
w = BuckarooWidget(df)
w

In [ ]:
ab = df['tripduration']
ab.sort_values()

In [ ]:
sorted_ab = ab.sort_values()
sorted_ab.index[:5]

In [ ]:
import numpy as np

In [ ]:
np.unique([2,2,3])

In [ ]:
df['start station name'].sort_values()

In [ ]:
np.array([2,2,3]).unique()

In [ ]:
a = []
a.extend([23,45], [32,5])
a

In [ ]:
df.columns

In [ ]:
bc = df.sample(300)

In [ ]:
bc.index

In [ ]:
arr = [2,3]
arr.extend(df.index)

In [ ]:
df.iloc[[137, 137, 138]]

In [ ]:
pd.concat([df.iloc[sorted_ab.index[:5]], df[20:50]])

In [ ]:
from buckaroo.all_transforms import Command
from buckaroo.lispy import s
#Here we start adding commands to the Buckaroo Widget.  Every call to add_command replaces a command with the same name
@w.add_command
class GroupBy2(Command):
    command_default = [s("groupby2"), s('df'), 'col', {}]
    command_pattern = [[3, 'colMap', 'colEnum', ['null', 'sum', 'mean', 'median', 'count']]]
    @staticmethod 
    def transform(df, col, col_spec):
        grps = df.groupby(col)
        df_contents = {}
        for k, v in col_spec.items():
            if v == "sum":
                df_contents[k] = grps[k].apply(lambda x: x.sum())
            elif v == "mean":
                df_contents[k] = grps[k].apply(lambda x: x.mean())
            elif v == "median":
                df_contents[k] = grps[k].apply(lambda x: x.median())
            elif v == "count":
                df_contents[k] = grps[k].apply(lambda x: x.count())
        return pd.DataFrame(df_contents)

    @staticmethod 
    def transform_to_py(df, col, col_spec):
        commands = [
            "    grps = df.groupby('%s')" % col,
            "    df_contents = {}"
        ]
        for k, v in col_spec.items():
            if v == "sum":
                commands.append("    paddydf_contents['%s'] = grps['%s'].apply(lambda x: x.sum())" % (k, k))
            elif v == "mean":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.mean())" % (k, k))
            elif v == "median":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.median())" % (k, k))
            elif v == "count":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.count())" % (k, k))
        commands.append("    df = pd.DataFrame(df_contents)")
        return "\n".join(commands)
